In [7]:
import os
import pandas as pd
# import lxml as lx
from lxml import html
from glob import glob

import sys, os

# Add the scripts/SEC_Documents folder to Python path
sys.path.append(os.path.abspath("scripts/SEC_Documents"))
# sys.path.append(os.path.abspath("/Volumes/SeagateX/Political-Economy-Of-Corporate-Fraud/data/A"))
from Deepseek_Integration import process_csv_with_deepseek

os.getcwd()

'/Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud/scripts/SEC_Documents'

In [11]:
folder_path = "/Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud/data/A"


# folder_path = "/Volumes/SeagateX/Political-Economy-Of-Corporate-Fraud/data/A"

files_name = glob(os.path.join(folder_path, "*.html"))
print(f"found {len(files_name)} files")
print(files_name[:5])

for file in files_name[:1]:
    print(f"Processing file: {file}")
    with open(file, "r") as f:
        content = f.read()
        try :
            tree = html.fromstring(content)
            xpath_expr = """
        //tr[
            .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'name')]
            and .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'principal')]
            and .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'position')]
        ]
        """
            tr_node = tree.xpath(xpath_expr)
            print(f"Found {len(tr_node)} matching <tr> nodes in {file}")
            if tr_node:
                table = tr_node[0].getparent()
                # print(f"Table: {html.tostring(table, pretty_print=True).decode('utf-8')}")
                while table is not None and table.tag != 'table':
                    table = table.getparent()
                print(f"Table: {html.tostring(table, pretty_print=True).decode('utf-8')}")

        except Exception as e:
            print(f"Error parsing {file}: {e}")


found 0 files
[]


In [9]:
# Compare types
# print("Type of table:", type(table))
# print("Type of lxml.html.tostring(table):", type(html.tostring(table)))

# Show what they look like
print("\nPrinting table directly:")
print(table)   # lxml object representation

print("\nPrinting tostring(table):")
print(html.tostring(table)[:300])  # raw HTML string (first 300 chars)

# Try parsing with pandas


Printing table directly:


NameError: name 'table' is not defined

In [6]:
df = pd.read_html(html.tostring(table))[0]
print("\nDataFrame head:")
print(df)


DataFrame head:
                                                  0   1                 2   \
0                                                NaN NaN               NaN   
1                                                NaN NaN               NaN   
2                        Name and Principal Position NaN              Year   
3  Steven A. Ballmer  Chief Executive Officer; Di... NaN  2005  2004  2003   
4  William H. Gates III  Chairman; Chief Software... NaN  2005  2004  2003   
5  Kevin R. Johnson  Co-President, Microsoft Plat... NaN  2005  2004  2003   
6  Jeffrey S. Raikes  President, Microsoft Busine... NaN  2005  2004  2003   
7  James E. Allchin  Co-President, Microsoft Plat... NaN  2005  2004  2003   

   3                    4                          5                    6   \
0 NaN                  NaN                        NaN                  NaN   
1 NaN  Annual Compensation        Annual Compensation  Annual Compensation   
2 NaN               Salary                    

In [7]:
# df.columns = [
#     " ".join([str(x) for x in col if x and "Unnamed" not in str(x)]).strip()
#     for col in df.columns.values
# ]

df.columns = [
    " ".join(
        [str(x) for x in (col if isinstance(col, tuple) else [col]) if x and "Unnamed" not in str(x)]
    ).strip()
    for col in df.columns.values
]

df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else x)

print("\nCleaned DataFrame head:")
print(df)


Cleaned DataFrame head:
                                                       1                 2  \
0                                                NaN NaN               NaN   
1                                                NaN NaN               NaN   
2                        Name and Principal Position NaN              Year   
3  Steven A. Ballmer  Chief Executive Officer; Di... NaN  2005  2004  2003   
4  William H. Gates III  Chairman; Chief Software... NaN  2005  2004  2003   
5  Kevin R. Johnson  Co-President, Microsoft Plat... NaN  2005  2004  2003   
6  Jeffrey S. Raikes  President, Microsoft Busine... NaN  2005  2004  2003   
7  James E. Allchin  Co-President, Microsoft Plat... NaN  2005  2004  2003   

    3                    4                          5                    6  \
0 NaN                  NaN                        NaN                  NaN   
1 NaN  Annual Compensation        Annual Compensation  Annual Compensation   
2 NaN               Salary            

/var/folders/wq/snfnd7nx5gxb7f385g58spl00000gn/T/ipykernel_10195/4280171313.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if pd.notna(x) else x)


In [8]:
df = df.dropna(axis=1, how='all')  # drop columns where all elements are NaN

print(df)

                                                                     2  \
0                                                NaN               NaN   
1                                                NaN               NaN   
2                        Name and Principal Position              Year   
3  Steven A. Ballmer  Chief Executive Officer; Di...  2005  2004  2003   
4  William H. Gates III  Chairman; Chief Software...  2005  2004  2003   
5  Kevin R. Johnson  Co-President, Microsoft Plat...  2005  2004  2003   
6  Jeffrey S. Raikes  President, Microsoft Busine...  2005  2004  2003   
7  James E. Allchin  Co-President, Microsoft Plat...  2005  2004  2003   

                     4                          5                    6  \
0                  NaN                        NaN                  NaN   
1  Annual Compensation        Annual Compensation  Annual Compensation   
2               Salary                     Salary                  NaN   
3                    $  600,000  591,

In [9]:
output_dir = os.path.join(folder_path, "extracted_jupiter_data")
os.makedirs(output_dir, exist_ok=True)  # create folder if it doesn't exist

output_path = os.path.join(output_dir, "2024_cleaned.csv")
df.to_csv(output_path, index=False)

print(f"Saved cleaned table to {output_path}")

Saved cleaned table to /Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud/data/MSFT/extracted_jupiter_data/2024_cleaned.csv


In [10]:
# Path to your saved cleaned CSV
output_path = os.path.join(folder_path, "extracted_jupiter_data", "1999_cleaned.csv")

# Call DeepSeek on it
process_csv_with_deepseek(output_path, "Convert this SEC Summary Compensation Table into structured JSON")


DeepSeek Response:
 Thinking...
Alright, let's tackle this problem step by step. The user has provided an SEC Summary Compensation Table that needs to be converted into structured JSON. I need to make sure I understand the structure of both the table and the desired output.

First, looking at the table, it seems a bit messy with multiple entries for each person across different years. Each row represents a year for a specific name and position. The columns include Year, Annual Compensation, Salary, Bonus, Long-Term Compensation Awards, Securities Underlying Options, and All Other Compensation.

I notice that the same person appears multiple times, once per year, which suggests that each entry should be treated as a separate object within an array for that individual. So, for example, William H. Gates has entries for 1997, 1998, and 1999, each with their own compensation details.

Next, I need to structure the JSON accordingly. The top-level keys should likely be "name" and "principal_